In [12]:
# Import libraries
import pandas as pd
import numpy as np
import boto3
import mysql.connector
# this will try to make a connection to a database in AWS
try:
    mydb = mysql.connector.connect(
      host="localhost", # URL or hostname of the database in Oregon  
      user="admin",
      password="34FEJe?jq34!!0f",
      database='rwdp')
# if an error occurs it will print out the problem    
except mysql.connector.Error as err:
    print("Something went wrong: {}".format(err))

mycursor = mydb.cursor()

### Customize this line per file type
query = "SELECT FileLocation from rwdp.AsosMeasurements"


mycursor.execute(query)
myresult = mycursor.fetchall()

headers = ['FileLocation']

dbfiles_df = pd.DataFrame(data = myresult, 
                 columns = headers)
# 6. Close the database
# It's# 6. Close the database good practice to close the database connection once you are done pulling data    
mydb.close()

dbfiles_df.head()

,FileLocation
0,asos/Asos.0604.20191201000029.csv
1,asos/Asos.0604.20191201000130.csv
2,asos/Asos.0604.20191201000230.csv
3,asos/Asos.0604.20191201000329.csv
4,asos/Asos.0604.20191201000430.csv


In [11]:
import boto3
session = boto3.Session(
    region_name='us-gov-west-1',
    profile_name='vafb'
)
s3 = session.resource('s3')
bucket = s3.Bucket('dxhub-vafb-xui-weather-data-raw')
## List objects within a given prefix
s3files = []
for obj in bucket.objects.filter(Delimiter='/', Prefix='asos/'):
    s3files.append(obj.key)

s3files_df = pd.DataFrame(data = s3files, 
                 columns = headers)

asos/Asos.0603.20191231000029.csv
asos/Asos.0603.20191231000129.csv


In [14]:
# assign location to each dataframe that will we use in the merge
s3files_df['location'] = 's3'
dbfiles_df['location'] = 'db'


In [15]:
print (s3files_df.shape)
print (dbfiles_df.shape)
s3files_df.head()
#dbfiles_df.head()

(1138089, 2)
(914617, 2)


,FileLocation,location
0,asos/Asos.0603.20191231000029.csv,s3
1,asos/Asos.0603.20191231000129.csv,s3
2,asos/Asos.0603.20191231000229.csv,s3
3,asos/Asos.0603.20191231000329.csv,s3
4,asos/Asos.0603.20191231000429.csv,s3


In [16]:
# We want a list of files that are on S3 but NOT in the DB, those files need to be deleted
# Let's do a left merge so that any files cleans will show up as NaN
files_merged = s3files_df.merge(dbfiles_df, on='FileLocation', how='left')


files_not_in_db_filter = files_merged['location_y'].isnull()
files_that_exist_on_s3_not_in_db = files_merged[files_not_in_db_filter]
print ('files_that_exist_on_s3_not_in_db', files_that_exist_on_s3_not_in_db.shape)
files_that_exist_on_s3_not_in_db.head()

files_that_exist_on_s3_not_in_db (223472, 3)


,FileLocation,location_x,location_y
1505,asos/Asos.0603.20200201010529.csv,s3,NaN
1513,asos/Asos.0603.20200201011329.csv,s3,NaN
1515,asos/Asos.0603.20200201011529.csv,s3,NaN
1520,asos/Asos.0603.20200201012029.csv,s3,NaN
1528,asos/Asos.0603.20200201012829.csv,s3,NaN


In [17]:
files_that_exist_on_s3_not_in_db.iloc[2]['FileLocation']

'asos/Asos.0603.20200201011529.csv'

In [ ]:
# Use the SDK to remove all files from S3 not in the DB
session = boto3.Session(
    region_name='us-gov-west-1',
    profile_name='vafb'
)

client = session.client('s3')
for objects in files_that_exist_on_s3_not_in_db.iterrows():
    key = objects[1]['FileLocation']
    response = client.delete_object(Bucket='dxhub-vafb-xui-weather-data-raw',Key=key)